### Notebook for concept detection in neural network

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import os
import sys
from tqdm import tqdm
import json
import random
import string

sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))

import concepts
import env
from policy import ActorCriticNet

model_name = "net"
session_name = "starship"
board_size = 5
board_name = f'{board_size}x{board_size}'

agents_to_sample = [0, 150, 300, 450, 600, 750, 900, 1050]

full_model_path = f"../models/saved_sessions/board_size_{board_size}/{session_name}/"

CONCEPT_FUNC = concepts.concept_one_eye

CONCEPT_NAME = concepts.concept_one_eye.__name__


2023-10-26 10:04:14.649648: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
def load_model(full_name, model_name, epoch):
    model_path = full_name + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, model_path)
    return model

agents = [load_model(full_model_path, model_name, epoch) for epoch in agents_to_sample]

In [10]:
def play_match(agents: list[ActorCriticNet], board_size, concept_function):
    go_env = env.GoEnv(board_size)
    state = go_env.reset()

    player_to_start = 1 if np.random.random() > 0.5 else 0
    current_player = player_to_start

    total_moves = board_size * board_size * 4
    moves = 0

    positive_cases = []
    negative_cases = []

    sample_ratio = 0.2

    game_over = False

    while not game_over:
        if moves == total_moves:
            break
        # Remove array index 3 and 5 from the current state making it an shape of (4, 5, 5)
        state_copy = np.delete(state, [3, 5], axis=0)
        
        if np.random.random() < sample_ratio:
            if concept_function(state):
                positive_cases.append(state_copy)
            else:
                negative_cases.append(state_copy)
        
        action = agents[current_player].best_action(state)

        state, _, game_over, _ = go_env.step(action)
        moves += 1

        current_player = (current_player + 1) % 2
    
    return positive_cases, negative_cases

positive_cases = []
negative_cases = []

CASES_TO_SAMPLE = 2500 # 25000

positive_bar = tqdm(total=CASES_TO_SAMPLE, desc="Positive cases")

while len(positive_cases) < CASES_TO_SAMPLE and len(negative_cases) < CASES_TO_SAMPLE:
    for i in range(len(agents)):
        for j in range(i + 1, len(agents)):
            pos, neg = play_match([agents[i], agents[j]], board_size, CONCEPT_FUNC)
            positive_cases.extend(pos)
            negative_cases.extend(neg)
            positive_bar.update(len(pos))

positive_cases = positive_cases[:CASES_TO_SAMPLE]
negative_cases = negative_cases[:CASES_TO_SAMPLE]

# Create numpy arrays
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

Positive cases:   3%|▎         | 77/2500 [00:08<04:25,  9.14it/s]
Positive cases: 2577it [02:04, 24.56it/s]                          

Positive cases: 2577it [02:21, 24.56it/s]

In [11]:
# Print shapes
print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (2500, 4, 5, 5)
Negative cases:  (2037, 4, 5, 5)


In [ ]:
# Find how many positive cases are identical to other positive cases and count the number of toatal cases
identical_cases = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            identical_cases += 1

print(f"Total cases: {len(positive_cases)}")
print(f"Identical cases: {identical_cases}")

In [ ]:
# Print all the positive cases
for i in range(len(positive_cases)):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print()

In [32]:
# Positions to consider are 80% of the total positions
POSITIONS_TO_CONSIDER = int(0.8 * positive_cases.shape[0] * 2)
print(f"Positions to consider: {POSITIONS_TO_CONSIDER}")
#POSITIONS_TO_CONSIDER = 4000 #40000
VALIDATION_POSITIONS = 10000 #10000

BINARY = True

Positions to consider: 4000


In [36]:
# First test if the concept can be regressed form the inputs
name = "input"
all_cases = np.concatenate([positive_cases, negative_cases])
all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
all_labels = np.array(all_labels)
shuffled_indices = np.arange(all_labels.shape[0])

np.random.shuffle(shuffled_indices)

all_cases = all_cases[shuffled_indices]
all_labels = all_labels[shuffled_indices]

points = all_cases.reshape(all_cases.shape[0], -1)

# Use the regression
score = concepts.perform_regression(
    points=points[:POSITIONS_TO_CONSIDER], 
    targets=all_labels[:POSITIONS_TO_CONSIDER], 
    validation_points=points[POSITIONS_TO_CONSIDER:], 
    validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
    is_binary=BINARY
)

print("Regression score: ", score)

# Remove the files if they exist
if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
    # Test if epoch folder exists
    if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
        # Remove all files in the epoch folder
        # Fist save the 
        for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
            os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name, file))

os.makedirs("../concept_presences", exist_ok=True)
os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name), exist_ok=True)

random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

# Save concept presences in json file
with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, name, random_suffix), "w") as f:
    json.dump(score, f)

Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 0.7263 - val_loss: 0.7208
Epoch 2/50
125/125 [==============================] - 0s 930us/step - loss: 0.7047 - val_loss: 0.7004
Epoch 3/50
125/125 [==============================] - 0s 897us/step - loss: 0.6896 - val_loss: 0.6849
Epoch 4/50
125/125 [==============================] - 0s 881us/step - loss: 0.6767 - val_loss: 0.6722
Epoch 5/50
125/125 [==============================] - 0s 890us/step - loss: 0.6652 - val_loss: 0.6604
Epoch 6/50
125/125 [==============================] - 0s 888us/step - loss: 0.6544 - val_loss: 0.6493
Epoch 7/50
125/125 [==============================] - 0s 941us/step - loss: 0.6442 - val_loss: 0.6390
Epoch 8/50
125/125 [==============================] - 0s 926us/step - loss: 0.6348 - val_loss: 0.6299
Epoch 9/50
125/125 [==============================] - 0s 893us/step - loss: 0.6262 - val_loss: 0.6211
Epoch 10/50
125/125 [==============================] - 0s 960us/step - loss: 0.6179 

In [34]:
epochs_to_look_at = [0, 150, 300, 450, 600, 750, 900, 1050]

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_resblock_outputs(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = concepts.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=BINARY
        )
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    # Remove the files if they exist
    if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
        # Test if epoch folder exists
        if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
            # Remove all files in the epoch folder
            # Fist save the 
            for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
                os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch, file))

    os.makedirs("../concept_presences", exist_ok=True)
    os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch), exist_ok=True)

    random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

    # Save concept presences in json file
    with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, epoch, random_suffix), "w") as f:
        json.dump(concept_presences[CONCEPT_NAME], f)




Getting resblock outputs: 100%|██████████| 142/142 [00:00<00:00, 291.41it/s]


Output shape:  (4537, 4, 5, 32)
Performing regression for layer 0
Points shape:  (4537, 640)
Epoch 1/50
125/125 [==============================] - 1s 2ms/step - loss: 0.9603 - val_loss: 0.9327
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8984 - val_loss: 0.8765
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8459 - val_loss: 0.8280
Epoch 4/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8005 - val_loss: 0.7866
Epoch 5/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7623 - val_loss: 0.7518
Epoch 6/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7310 - val_loss: 0.7237
Epoch 7/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7055 - val_loss: 0.7006
Epoch 8/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6851 - val_loss: 0.6825
Epoch 9/50
125/125 [==============================] - 0s 974us/step - loss: 0.6703 - val_loss: 0.6713
Epoch

Getting resblock outputs: 100%|██████████| 142/142 [00:00<00:00, 270.18it/s]


Output shape:  (4537, 4, 5, 32)
Performing regression for layer 0
Points shape:  (4537, 640)
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 0.9225 - val_loss: 0.8665
Epoch 2/50
125/125 [==============================] - 0s 914us/step - loss: 0.8275 - val_loss: 0.7946
Epoch 3/50
125/125 [==============================] - 0s 886us/step - loss: 0.7684 - val_loss: 0.7439
Epoch 4/50
125/125 [==============================] - 0s 881us/step - loss: 0.7237 - val_loss: 0.7049
Epoch 5/50
125/125 [==============================] - 0s 853us/step - loss: 0.6876 - val_loss: 0.6694
Epoch 6/50
125/125 [==============================] - 0s 876us/step - loss: 0.6566 - val_loss: 0.6398
Epoch 7/50
125/125 [==============================] - 0s 873us/step - loss: 0.6296 - val_loss: 0.6142
Epoch 8/50
125/125 [==============================] - 0s 874us/step - loss: 0.6057 - val_loss: 0.5926
Epoch 9/50
125/125 [==============================] - 0s 913us/step - loss: 0.5853 - val_loss

Getting resblock outputs: 100%|██████████| 142/142 [00:00<00:00, 283.58it/s]


...

Output shape:  (4537, 4, 5, 32)
Performing regression for layer 0
Points shape:  (4537, 640)
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 1.0327 - val_loss: 0.9136
Epoch 2/50
125/125 [==============================] - 0s 901us/step - loss: 0.8580 - val_loss: 0.8011
Epoch 3/50
125/125 [==============================] - 0s 883us/step - loss: 0.7641 - val_loss: 0.7306
Epoch 4/50
125/125 [==============================] - 0s 888us/step - loss: 0.7017 - val_loss: 0.6799
Epoch 5/50
125/125 [==============================] - 0s 888us/step - loss: 0.6546 - val_loss: 0.6402
Epoch 6/50
125/125 [==============================] - 0s 917us/step - loss: 0.6176 - val_loss: 0.6076
Epoch 7/50
125/125 [==============================] - 0s 879us/step - loss: 0.5869 - val_loss: 0.5801
Epoch 8/50
125/125 [==============================] - 0s 891us/step - loss: 0.5604 - val_loss: 0.5563
Epoch 9/50
125/125 [==============================] - 0s 890us/step - loss: 0.5373 - val_loss

Getting resblock outputs: 100%|██████████| 142/142 [00:00<00:00, 255.48it/s]


Output shape:  (4537, 4, 5, 32)
Performing regression for layer 0
Points shape:  (4537, 640)
Epoch 1/50
125/125 [==============================] - 1s 2ms/step - loss: 1.0079 - val_loss: 0.9178
Epoch 2/50
125/125 [==============================] - 0s 916us/step - loss: 0.8729 - val_loss: 0.8200
Epoch 3/50
125/125 [==============================] - 0s 902us/step - loss: 0.7895 - val_loss: 0.7510
Epoch 4/50
125/125 [==============================] - 0s 888us/step - loss: 0.7283 - val_loss: 0.6966
Epoch 5/50
125/125 [==============================] - 0s 880us/step - loss: 0.6793 - val_loss: 0.6528
Epoch 6/50
125/125 [==============================] - 0s 897us/step - loss: 0.6391 - val_loss: 0.6169
Epoch 7/50
125/125 [==============================] - 0s 899us/step - loss: 0.6056 - val_loss: 0.5868
Epoch 8/50
125/125 [==============================] - 0s 883us/step - loss: 0.5776 - val_loss: 0.5610
Epoch 9/50
125/125 [==============================] - 0s 932us/step - loss: 0.5536 - val_loss

Getting resblock outputs: 100%|██████████| 142/142 [00:00<00:00, 287.83it/s]


Output shape:  (4537, 4, 5, 32)
Performing regression for layer 0
Points shape:  (4537, 640)
Epoch 1/50
125/125 [==============================] - 1s 2ms/step - loss: 1.0011 - val_loss: 0.8589
Epoch 2/50
125/125 [==============================] - 0s 951us/step - loss: 0.8131 - val_loss: 0.7442
Epoch 3/50
125/125 [==============================] - 0s 914us/step - loss: 0.7266 - val_loss: 0.6726
Epoch 4/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6673 - val_loss: 0.6197
Epoch 5/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6217 - val_loss: 0.5789
Epoch 6/50
125/125 [==============================] - 0s 931us/step - loss: 0.5853 - val_loss: 0.5466
Epoch 7/50
125/125 [==============================] - 0s 935us/step - loss: 0.5559 - val_loss: 0.5197
Epoch 8/50
125/125 [==============================] - 0s 896us/step - loss: 0.5318 - val_loss: 0.4982
Epoch 9/50
125/125 [==============================] - 0s 882us/step - loss: 0.5116 - val_loss: 0.